In [1]:
import os
from glob import glob
import pandas as pd
from functools import reduce
from xml.etree import ElementTree as et


In [2]:
#loading xml and storing it in list
xml_list = glob('./test/*.xml')
#data cleaning
xmlfiles = []
xmlfiles = list(map(lambda x:x.replace("\\","/"),xml_list))
xmlfiles

['./test/EYE DOWN_1.xml',
 './test/EYE DOWN_10.xml',
 './test/EYE DOWN_11.xml',
 './test/EYE DOWN_12.xml',
 './test/EYE DOWN_13.xml',
 './test/EYE DOWN_14.xml',
 './test/EYE DOWN_15.xml',
 './test/EYE DOWN_16.xml',
 './test/EYE DOWN_17.xml',
 './test/EYE DOWN_18.xml',
 './test/EYE DOWN_19.xml',
 './test/EYE DOWN_2.xml',
 './test/EYE DOWN_20.xml',
 './test/EYE DOWN_21.xml',
 './test/EYE DOWN_22.xml',
 './test/EYE DOWN_23.xml',
 './test/EYE DOWN_24.xml',
 './test/EYE DOWN_25.xml',
 './test/EYE DOWN_26.xml',
 './test/EYE DOWN_27.xml',
 './test/EYE DOWN_28.xml',
 './test/EYE DOWN_29.xml',
 './test/EYE DOWN_3.xml',
 './test/EYE DOWN_30.xml',
 './test/EYE DOWN_31.xml',
 './test/EYE DOWN_32.xml',
 './test/EYE DOWN_33.xml',
 './test/EYE DOWN_34.xml',
 './test/EYE DOWN_35.xml',
 './test/EYE DOWN_36.xml',
 './test/EYE DOWN_37.xml',
 './test/EYE DOWN_38.xml',
 './test/EYE DOWN_39.xml',
 './test/EYE DOWN_4.xml',
 './test/EYE DOWN_40.xml',
 './test/EYE DOWN_5.xml',
 './test/EYE DOWN_6.xml',
 './tes

In [3]:
#read xml file, extract filename, width, height, xmin, xmax, ymin, ymax
def extract_text(filename):
    tree = et.parse(filename)
    root = tree.getroot()
    image_name = root.find('filename').text
    width = root.find('size').find('width').text
    height = root.find('size').find('height').text
    objs = root.findall('object')
    parser = []
    for obj in objs:
        name = obj.find('name').text
        bndbox = obj.find('bndbox')
        xmin = bndbox.find('xmin').text
        xmax = bndbox.find('xmax').text
        ymin = bndbox.find('ymin').text
        ymax = bndbox.find('ymax').text
        parser.append([image_name,width,height,name,xmin,xmax,ymin,ymax])
    return parser
parser_all = list(map(extract_text,xmlfiles))
parser_all
extract_text('./test/EYE DOWN_10.xml')

[['EYE DOWN_10.jpg', '800', '600', 'EYE DOWN', '161', '256', '156', '270'],
 ['EYE DOWN_10.jpg', '800', '600', 'EYE DOWN', '501', '595', '193', '294']]

In [4]:
extract_text('./test/EYE DOWN_10.xml')

[['EYE DOWN_10.jpg', '800', '600', 'EYE DOWN', '161', '256', '156', '270'],
 ['EYE DOWN_10.jpg', '800', '600', 'EYE DOWN', '501', '595', '193', '294']]

In [5]:
data = reduce(lambda x,y : x+y, parser_all) 
    

In [6]:
df = pd.DataFrame(data,columns=['filename','width','height','name','xmin','xmax','ymin','ymax'])
df

,filename,width,height,name,xmin,xmax,ymin,ymax
0,EYE DOWN_1.jpg,800,600,EYE DOWN,101,279,367,494
1,EYE DOWN_1.jpg,800,600,EYE DOWN,571,725,403,521
2,EYE DOWN_10.jpg,800,600,EYE DOWN,161,256,156,270
3,EYE DOWN_10.jpg,800,600,EYE DOWN,501,595,193,294
4,EYE DOWN_11.jpg,800,600,EYE DOWN,204,284,227,308
...,...,...,...,...,...,...,...,...
311,EYE UP_7.jpg,800,600,EYE UP,502,644,281,379
312,EYE UP_8.jpg,800,600,EYE UP,127,307,280,420
313,EYE UP_8.jpg,800,600,EYE UP,565,705,234,372
314,EYE UP_9.jpg,800,600,EYE UP,131,256,278,369


In [7]:
df.shape

(316, 8)

In [8]:
df['name'].value_counts()

name
EYE DOWN     81
EYE RIGHT    80
EYE LEFT     79
EYE UP       76
Name: count, dtype: int64

In [9]:
cols = ['width', 'height', 'xmin', 'xmax', 'ymin', 'ymax']  # Exclude 'name' if it's a string
df[cols] = df[cols].astype(int)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316 entries, 0 to 315
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   filename  316 non-null    object
 1   width     316 non-null    int64 
 2   height    316 non-null    int64 
 3   name      316 non-null    object
 4   xmin      316 non-null    int64 
 5   xmax      316 non-null    int64 
 6   ymin      316 non-null    int64 
 7   ymax      316 non-null    int64 
dtypes: int64(6), object(2)
memory usage: 19.9+ KB


In [10]:
#center x, center y
df['center_x'] = ((df['xmin']+df['xmax'])/2)/df['width']
df['center_y'] = ((df['ymin']+df['ymax'])/2)/df['height']

In [11]:
#w #h
df['w'] = (df['xmax']-df['xmin'])/df['width']
df['h'] = (df['ymax']-df['ymin'])/df['height']

In [12]:
#label encoding
df.rename(columns={'name': 'class'}, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316 entries, 0 to 315
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   filename  316 non-null    object 
 1   width     316 non-null    int64  
 2   height    316 non-null    int64  
 3   class     316 non-null    object 
 4   xmin      316 non-null    int64  
 5   xmax      316 non-null    int64  
 6   ymin      316 non-null    int64  
 7   ymax      316 non-null    int64  
 8   center_x  316 non-null    float64
 9   center_y  316 non-null    float64
 10  w         316 non-null    float64
 11  h         316 non-null    float64
dtypes: float64(4), int64(6), object(2)
memory usage: 29.8+ KB


In [13]:
df['class'] = df['class'].str.replace(r'_\d+', '', regex=True)
df['class'].unique()

array(['EYE DOWN', 'EYE LEFT', 'EYE RIGHT', 'EYE UP'], dtype=object)

In [14]:

required_columns = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
df_selected = df[required_columns]

# Save the selected columns to a CSV file
df_selected.to_csv('test_label1.csv', index=False)